# **Text generation using Recurrent Neural Network NLP**

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import os
import datetime 
import numpy as np
import tensorflow as tf
from urllib.request import urlopen

In [ ]:
datafile_path = 'https://sherlock-holm.es/stories/plain-text/advs.txt'

In [ ]:
data = urlopen(datafile_path).read().decode('utf-8')
print(len(data))

610871


In [ ]:
print(data[:1000])





                        THE ADVENTURES OF SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Scandal in Bohemia
               The Red-Headed League
               A Case of Identity
               The Boscombe Valley Mystery
               The Five Orange Pips
               The Man with the Twisted Lip
               The Adventure of the Blue Carbuncle
               The Adventure of the Speckled Band
               The Adventure of the Engineer's Thumb
               The Adventure of the Noble Bachelor
               The Adventure of the Beryl Coronet
               The Adventure of the Copper Beeches










                              A SCANDAL IN BOHEMIA





                                Table of contents
                                     Chapter 1
                                     Chapter 2
                                     Chapter 3















          CHAPTER I



  

In [ ]:
text = data[1000:]

In [ ]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

84 unique characters


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print('  {:4s}: {:3d}'.format(repr(char), char2idx[char]))
print('....\n}')

{
  '\n':   0
  ' ' :   1
  '!' :   2
  '"' :   3
  '&' :   4
  "'" :   5
  '(' :   6
  ')' :   7
  ',' :   8
  '-' :   9
  '.' :  10
  '/' :  11
  '0' :  12
  '1' :  13
  '2' :  14
  '3' :  15
  '4' :  16
  '5' :  17
  '6' :  18
  '7' :  19
....
}


In [ ]:
print('{} ----char-2-int-----{}'.format(repr(text[40:60]), text_as_int[40:60]))

'woman. I have seldom' ----char-2-int-----[74 66 64 52 65 10  1 33  1 59 52 73 56  1 70 56 63 55 66 64]


In [ ]:
seq_length = 100
exmaples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

 
 
 
T
o
 
S
h
e
r


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))
  print("-"*110)

'   To Sherlock Holmes she is always the woman. I have seldom heard him\n     mention her under any oth'
--------------------------------------------------------------------------------------------------------------
'er name. In his eyes she eclipses and\n     predominates the whole of her sex. It was not that he felt'
--------------------------------------------------------------------------------------------------------------
' any\n     emotion akin to love for Irene Adler. All emotions, and that one\n     particularly, were ab'
--------------------------------------------------------------------------------------------------------------
'horrent to his cold, precise but admirably\n     balanced mind. He was, I take it, the most perfect re'
--------------------------------------------------------------------------------------------------------------
'asoning and\n     observing machine that the world has seen, but as a lover he would\n     have placed '
-----------------------------

Split the input and ground truth

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Visualize Input and Ground Truth

In [ ]:
for input_example, target_example in dataset.take(1):
  print('Input data:', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data: '   To Sherlock Holmes she is always the woman. I have seldom heard him\n     mention her under any ot'
Target data: '  To Sherlock Holmes she is always the woman. I have seldom heard him\n     mention her under any oth'


Expection from Model

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("step {:4d}".format(i))
  print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print(" expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

step    0
 input: 1 (' ')
 expected output: 1 (' ')
step    1
 input: 1 (' ')
 expected output: 1 (' ')
step    2
 input: 1 (' ')
 expected output: 44 ('T')
step    3
 input: 44 ('T')
 expected output: 66 ('o')
step    4
 input: 66 ('o')
 expected output: 1 (' ')


Prepare Training Batch

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [ ]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Dataset Shape={}".format(dataset))

Dataset Shape=<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


Prepare Model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):


  model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
  tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
  tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
vocab_size = len(vocab)

embedding_dim = 512

rnn_units = 1024

In [ ]:
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 512)           43008     
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          4724736   
_________________________________________________________________
dense_5 (Dense)              (64, None, 84)            86100     
Total params: 4,853,844
Trainable params: 4,853,844
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = r'data/training_chechpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tesnsorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
EPOCHS = 64
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, tesnsorboard_callback])

Epoch 1/64
94/94 [==============================] - 6s 67ms/step - loss: 2.8087
Epoch 2/64
94/94 [==============================] - 6s 65ms/step - loss: 1.9869
Epoch 3/64
94/94 [==============================] - 6s 65ms/step - loss: 1.7517
Epoch 4/64
94/94 [==============================] - 6s 65ms/step - loss: 1.5939
Epoch 5/64
94/94 [==============================] - 6s 66ms/step - loss: 1.4692
Epoch 6/64
94/94 [==============================] - 6s 66ms/step - loss: 1.3762
Epoch 7/64
94/94 [==============================] - 6s 67ms/step - loss: 1.3061
Epoch 8/64
94/94 [==============================] - 6s 67ms/step - loss: 1.2498
Epoch 9/64
94/94 [==============================] - 6s 67ms/step - loss: 1.2039
Epoch 10/64
94/94 [==============================] - 6s 69ms/step - loss: 1.1643
Epoch 11/64
94/94 [==============================] - 6s 68ms/step - loss: 1.1278
Epoch 12/64
94/94 [==============================] - 6s 68ms/step - loss: 1.0918
Epoch 13/64
94/94 [==================

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'data/training_chechpoints/ckpt_64'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 512)            43008     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
dense_6 (Dense)              (1, None, 84)             86100     
Total params: 4,853,844
Trainable params: 4,853,844
Non-trainable params: 0
_________________________________________________________________


context_string: Input string which acts as context for the model
num_generator: Number of Characters genearted
temperature: prameter to control rendomness of the outputs

return context string and test generated

In [ ]:
def generate_text(model, context_string, num_generate=1000, temperature=1.0):
  input_eval = [char2idx[s] for s in context_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (context_string + ''.join(text_generated))


In [ ]:
print(generate_text(model, context_string=u"Watson you are", num_generate=100, temperature=0.5))

Watson you are an hour the miserable weather and the
     moment the room and saw the door, "yet there not been in
